In [1]:
import sys
if 'E:\\jupyter_notebook\\HomemadeSEO_Phil\\' not in sys.path:
    sys.path.append('E:\\jupyter_notebook\\HomemadeSEO_Phil\\')
from ptl import get
get.setup(debug=True, driver=False)

> LOADED module:	get
> PLATFORM:	win32
> DEBUG:	True
> LIB_PATH:	E:\jupyter_notebook\HomemadeSEO_Phil\ptl\
> ROOT_PATH:	E:\jupyter_notebook\HomemadeSEO_Phil\Crawler
> START_TIME:	20171110_0933


In [195]:
#check for non www version

False

In [6]:
import numpy
get.N_FAIL_REQUEST=1
def has(a, alist):
    for item in alist:
        if item in a: return False
    return True
def get_hrefs(stack, url):
    hrefs=[]
    ROOT = get.dmr(url).split('/')[-1]
    filters=[' ', '?', 'javascript', 'tel:','fax:','mailto:', '.jpg', '.png']
    for row in stack:
        if 'SOUP' in row and row['SOUP']:
            _hrefs=[a['href']
                    for a in get.bes(None, div='a', pr=row['SOUP']) 
                    if a.has_attr('href')]
            _hrefs=[a
                    for a in _hrefs
                    if len(a)>3
                    and has(a, filters)]
            for a in _hrefs:
                if 'http' not in a:
                    _hrefs[_hrefs.index(a)]=(url+a)
            _hrefs=[a.replace('//','/').replace(':/', '://')
                    for a in _hrefs
                   if ROOT in a]
            hrefs=hrefs+distinct(_hrefs)
    return hrefs
def soups(rows, folder, t_id):
    for row in rows:
        try:    
            get.log('> GET '+row['URL'])
            rq = get.get(row['URL'])
            folders=row['URL'].split('//')[1].split('/')[:-1]
            for i, folder in enumerate(folders):
                if i==0: continue
                row['FOLDER LEVEL '+str(i)]=folder
            row['DEPTH'] = len(folders)
            row['FINAL URL']=rq.url
            row['FINAL URL LENGTH']=len(rq.url)
            row['HEADERS']=rq.headers
            row['CONTENT TYPE']=rq.headers['Content-Type'] if 'Content-Type' in rq.headers else ''
            #row['CONTENT LENGTH']=rq.headers['Content-Length'] if 'Content-Length' in rq.headers else ''
            #row['SERVER']=rq.headers['Server']
            row['DATE']=rq.headers['Date']
            #row['ENCODE']=rq.encoding
            row['RESPONSE TIME (MS)']=rq.elapsed.microseconds/1000
            row['REDIRECT TYPE']='NONE' if not rq.is_redirect else 'PERMANENT' if rq.is_permanent_redirect else 'TEMPERARY'
            row['REDIRECT']=rq.is_redirect
            row['STATUS']=rq.reason
            row['STATUS CODE']=rq.status_code
            row['HTTP/HTTPS']='HTTPS' if 'https' in rq.url[:10] else 'HTTP'
            row['SOUP'] = get.bs(rq.content, 'html.parser')
            check('title', row)
            check('h1', row)
            check('h2', row)
            check_meta(row)
            check_img(row)
            check_canonical_link(row)
        except: get.pe()
def check_length_status(_len, tag):
    if tag=='title':
        if _len == 0:  return 'BAD, MISSING '+tag.upper()
        if _len > 90: return 'TOO LONG'
        if _len > 70:  return 'LONG'
        if _len < 50:  return 'SHORT'
        if _len < 30:  return 'TOO SHORT'
        return 'GOOD'
    if tag=='description':
        if _len == 0:  return 'BAD, MISSING '+tag.upper()
        if _len > 160: return "TOO LONG"
        if _len < 40:  return 'TOO SHORT'
        return 'GOOD'
def check_img(row):
    try:
        imgs=row['SOUP'].findAll('img')
        imgs_alt_tag=[img['alt'] for img in imgs if img.has_attr('alt')]
        row['Number of images'.upper()] = len(imgs)
        row['Number of with alt-tag'.upper()] = len(imgs_alt_tag)
        row['Number of without alt-tag'.upper()] = len(imgs) - len(imgs_alt_tag)
        alt_tags=numpy.array([len(img) for img in imgs_alt_tag])
        if len(alt_tags)>0:
            row['alt-tag Minimum length'.upper()] = alt_tags.min()
            row['alt-tag Average length'.upper()] = alt_tags.mean()
            row['alt-tag Maximum length'.upper()] = alt_tags.max()
        df = get.DataFrame(imgs_alt_tag)
        row['Number of Alt-Tags that are duplicated'.upper()] = len(df[df.duplicated])       
    except: get.pe()
def check_meta(row):
    try:
        for i, meta in enumerate(row['SOUP'].findAll('meta')):
            if not meta.has_attr('name') or not meta.has_attr('content'): continue
            key = meta['name'].lower()
            if key!='description' and key!='keywords': continue
            key='META '+key.upper()
            row[key] = meta['content']
            _len = len(row[key])
            row[key+' LENGTH'] = _len
            #if key=='keywords':
                #row[key+' STATUS'] = check_length_status(_len, 'keywords')
            if key=='description':
                row[key+' STATUS'] = check_length_status(_len, 'description')       
    except: get.pe(str(meta))
def check_canonical_link(row):
    try:
        for i, link in enumerate(row['SOUP'].findAll('link')):
            if not link.has_attr('rel'): continue
            for i in link['rel']:
                if i.lower()=='canonical': 
                    key=i.upper()
                    row[key] = link['href']
                    row[key+' LENGTH'] = len(row[key])
                    break
            if 'CANONICAL' not in row:
                row['CANONICAL']='None'
                row['CANONICAL LENGTH']=''       
    except: get.pe(str(link))
def check(tag, row):
    try:
        for i, _tag in enumerate(row['SOUP'].findAll(tag)):
            key=tag.upper()
            if 'h' in tag: key=key+'-'+str(i+1)
            row[key]=_tag.text
            _len = len(row[key])
            row[key+' LENGTH'] = _len
            row[key+' STATUS'] = check_length_status(_len, tag)       
    except: get.pe(str(_tag))
def distinct(alist):
    return list(dict.fromkeys(alist))

get.START_TIME=get.dtn()
URL='https://www.truefit.com/'
DOMAIN = get.dmn(URL)
DOWNLOADED=[]
try:
    robots = get.get(URL+'robots.txt')
    if not robots:
        get.log('> robots.txt not found')
        DOWNLOADED.append({'URL': URL+'robots.txt', 'TITLE': 'Missing Robots.txt'})
    else:
        DOWNLOADED.append({'URL': URL+'robots.txt', 'TITLE': 'Found Robots.txt'})
except: 
    get.log('> robots.txt not found')
    get.pe()
VISISTED=[]
STACK=[URL]
N_THREADS= 5
THRESHOLD= 3 
ITER=1
while len(STACK)!=0:
    get.log('> Iteration:  '+str(ITER)+', stack size: '+str(len(STACK)))
    get.log('> Downloaded: '+str(len(DOWNLOADED)))
    stack=[{'URL':url} for url in STACK if '.pdf' not in url and '/members/' not in url]
    #print('> test'+str(get.split(STACK, N_THREADS)))
    if len(stack)/N_THREADS > THRESHOLD:
        N_THREADS=get.ceil(len(stack)/THRESHOLD)
    get.log('> Getting content')
    #soups(stack, '', '')
    get.run_threads(get.split(stack, N_THREADS), soups, folder='nodriver')
    VISISTED=VISISTED+STACK
    hrefs = get_hrefs(stack, URL)
    get.log('> New links:  '+str(len(hrefs)))
    DOWNLOADED=DOWNLOADED+stack
    STACK = list(set(hrefs)-set(hrefs).intersection(set(VISISTED)))
    get.log('> New stack: '+str(len(STACK)))
    STACK = STACK[:3]
    ITER+=1
    if ITER==3: break
get.log('> Finished after '+str(ITER)+' iterations')
DOWNLOADED=get.DataFrame(DOWNLOADED).sort_values('TITLE LENGTH', 
                                                 ascending=False)
columns=['URL',
        'FINAL URL',
        'FINAL URL LENGTH',
        'CANONICAL',
        'CANONICAL LENGTH',
        'TITLE',
        'TITLE LENGTH',
        'TITLE STATUS',
        'H1-1',
        'H1-1 LENGTH',
        'H1-1 STATUS',
        'META DESCRIPTION',
        'META DESCRIPTION LENGTH',
        'META DESCRIPTION STATUS',
        'META KEYWORDS',
        'META KEYWORDS LENGTH',
        'H2-1',
        'H2-1 LENGTH',
        'H2-1 STATUS',
        'NUMBER OF IMAGES',
        'NUMBER OF WITH ALT-TAG',
        'NUMBER OF WITHOUT ALT-TAG',
        'ALT-TAG AVERAGE LENGTH',
        'ALT-TAG MAXIMUM LENGTH',
        'ALT-TAG MINIMUM LENGTH',
        'NUMBER OF ALT-TAGS THAT ARE DUPLICATED',
        'CONTENT TYPE',
        'DATE',
        'HTTP/HTTPS',
        'REDIRECT',
        'REDIRECT TYPE',
        'RESPONSE TIME (MS)',
        'STATUS',
        'STATUS CODE',
        'DEPTH',
        'FOLDER LEVEL 1',
        'FOLDER LEVEL 2',
        'FOLDER LEVEL 3',]
get.save(DOWNLOADED, 
         DOMAIN+'_crawl',
         columns=columns)
def check_dp(df, output, col):
    try:
        if col in df.keys():
            dps=list(df[df.duplicated(col)].groupby(col))
            if len(dps)>0:
                for i, dp in enumerate(dps):
                    row_text, _dp = dp
                    _dp = df[df[col]==row_text]
                    row={}
                    row['TAG']=col
                    row['DUPLICATION TEXT'] = row_text
                    row['NUMBER OF DUPLICATION'] = len(_dp)
                    row['URL'] = '#\n'+'\n'.join(list(_dp['URL']))
                    output.append(row)
    except: get.pe(col)
DUPLICATIONS=[]
check_dp(DOWNLOADED, DUPLICATIONS, 'TITLE')
check_dp(DOWNLOADED, DUPLICATIONS, 'H1-1')
check_dp(DOWNLOADED, DUPLICATIONS, 'META DESCRIPTION')
check_dp(DOWNLOADED, DUPLICATIONS, 'H2-1')
def reformat_dp(dp, rows_text):
    dp = get.DataFrame(dp).sort_values(['TAG',
                                        'NUMBER OF DUPLICATION'],
                                       ascending=False)
    rs=get.DataFrame()
    for tag in rows_text:
        rs = get.concat([rs, dp[dp['TAG']==tag]])
    return rs

get.save(reformat_dp(DUPLICATIONS, 
                     ['TITLE', 'H1-1', 'META DESCRIPTION', 'H2-1']), 
         DOMAIN+'_duplications',
         columns=['TAG', 
                  'DUPLICATION TEXT',
                  'NUMBER OF DUPLICATION', 
                  'URL', ])
get.log('> Start time:'+get.START_TIME)
get.log('> Finish time:'+get.dtn())

> Iteration:  1, stack size: 1
> Downloaded: 1
> GET https://www.truefit.com/> Getting content

> Started multi threading:	20171110_0945
> Number of threads:      	1
> Finished multi threading:	20171110_0945
> New links:  31
> GET https://www.truefit.com/de/home> New stack: 31

> Iteration:  2, stack size: 3
> Downloaded: 2
> Getting content
> Started multi threading:	20171110_0945
> Number of threads:      	1
> GET https://www.truefit.com/about-us/careers
> GET https://www.truefit.com/en/Privacy-Policy
> Finished multi threading:	20171110_0945
> New links:  93
> New stack: 43
> Finished after 3 iterations
> Start time:20171110_0945
> Finish time:20171110_0945


In [215]:
df = get.DataFrame(DUPLICATIONS).sort_values(['TAG','NUMBER OF DUPLICATION'],
                                        ascending=False)

In [220]:
def reformat_dp(dp, rows_text):
    dp = get.DataFrame(dp).sort_values(['TAG',
                                        'NUMBER OF DUPLICATION'],
                                       ascending=False)
    rs=get.DataFrame()
    for tag in rows_text:
        rs = get.concat([rs, dp[dp['TAG']==tag]])
    return rs
reformat_dp(DUPLICATIONS, ['TITLE', 'H1-1', 'META DESCRIPTION', 'H2-1'])

,DUPLICATION TEXT,NUMBER OF DUPLICATION,TAG,URL
1,Car Detailing Chicago | Mobile Detailing Illin...,9,TITLE,#\nhttps://mobiledetailingpros.com/\nhttps://m...
0,Car Detailing - Keep Your Chicago Car Sparklin...,4,TITLE,#\nhttps://www.mobiledetailingpros.com/product...
10,Pick Up Truck - Our Detailers Offer Service Fo...,4,TITLE,#\nhttp://www.mobiledetailingpros.com/product/...
6,Mini Van - Our Car Detailing Can Keep Your Min...,3,TITLE,#\nhttps://mobiledetailingpros.com/product/min...
2,Closed Bow Boat / Speed Boat -,2,TITLE,#\nhttps://www.mobiledetailingpros.com/product...
3,Horse Trailer -,2,TITLE,#\nhttps://mobiledetailingpros.com/product/hor...
4,Join -,2,TITLE,#\nhttp://www.mobiledetailingpros.com/account/...
5,Large ATV -,2,TITLE,#\nhttps://mobiledetailingpros.com/product/lar...
7,Motorcycles/ATV Archives -,2,TITLE,#\nhttps://mobiledetailingpros.com/product-cat...
8,My Account -,2,TITLE,#\nhttps://www.mobiledetailingpros.com/my-acco...


In [219]:
get.concat([df[df['TAG']=='TITLE'],df[df['TAG']=='H1-1']])

,DUPLICATION TEXT,NUMBER OF DUPLICATION,TAG,URL
1,Car Detailing Chicago | Mobile Detailing Illin...,9,TITLE,#\nhttps://mobiledetailingpros.com/\nhttps://m...
0,Car Detailing - Keep Your Chicago Car Sparklin...,4,TITLE,#\nhttps://www.mobiledetailingpros.com/product...
10,Pick Up Truck - Our Detailers Offer Service Fo...,4,TITLE,#\nhttp://www.mobiledetailingpros.com/product/...
6,Mini Van - Our Car Detailing Can Keep Your Min...,3,TITLE,#\nhttps://mobiledetailingpros.com/product/min...
2,Closed Bow Boat / Speed Boat -,2,TITLE,#\nhttps://www.mobiledetailingpros.com/product...
3,Horse Trailer -,2,TITLE,#\nhttps://mobiledetailingpros.com/product/hor...
4,Join -,2,TITLE,#\nhttp://www.mobiledetailingpros.com/account/...
5,Large ATV -,2,TITLE,#\nhttps://mobiledetailingpros.com/product/lar...
7,Motorcycles/ATV Archives -,2,TITLE,#\nhttps://mobiledetailingpros.com/product-cat...
8,My Account -,2,TITLE,#\nhttps://www.mobiledetailingpros.com/my-acco...


In [214]:
get.DataFrame(DUPLICATIONS).groupby('TAG')

In [125]:
get.soup('https://mobiledetailingpros.com/product-category/boats-watercraft/')

In [162]:
check('img', row)

In [184]:

get_hrefs(stack, URL)

['https://mobiledetailingpros.com/#login_popup',
 'https://mobiledetailingpros.com/#register_popup',
 'http://www.mobiledetailingpros.com/account/sign-in/',
 'http://www.mobiledetailingpros.com/provider/sign-in/',
 'http://www.mobiledetailingpros.com/account/join/',
 'http://www.mobiledetailingpros.com/provider/join/',
 'https://www.mobiledetailingpros.com/cart/',
 'https://www.mobiledetailingpros.com/',
 'https://mobiledetailingpros.com/#site-main-content',
 'https://www.mobiledetailingpros.com/about-us/',
 'https://www.mobiledetailingpros.com/faq/',
 'https://www.mobiledetailingpros.com/product-category/carstruckssuv/',
 'https://www.mobiledetailingpros.com/product-category/boats-watercraft/',
 'https://www.mobiledetailingpros.com/product-category/industrial-vehicles/',
 'https://www.mobiledetailingpros.com/product-category/rvstrailers/',
 'https://www.mobiledetailingpros.com/product-category/motorcyclesatv/',
 'https://www.mobiledetailingpros.com/reviews/',
 'https://www.mobiledetai

In [117]:

DUPLICATIONS=[]
check_dp(DOWNLOADED, DUPLICATIONS, 'TITLE')
check_dp(DOWNLOADED, DUPLICATIONS, 'H1-1')
check_dp(DOWNLOADED, DUPLICATIONS, 'META DESCRIPTION')
check_dp(DOWNLOADED, DUPLICATIONS, 'H2-1')
get.save(get.DataFrame(DUPLICATIONS).sort_values('NUMBER OF DUPLICATION', ascending=False), 
         DOMAIN+'_Duplications',
         columns=['TAG', 
                  'DUPLICATION TEXT',
                  'NUMBER OF DUPLICATION', 
                  'URL', ])

In [170]:
'a', 'b' in 'asb'

('a', True)

In [86]:
for i, _tag in enumerate(DOWNLOADED.iloc[-4]['SOUP'].findAll('link')):
    
    print(_tag)

<link href="favicon.ico" rel="shortcut icon"/>
<link href="style/style.css" rel="stylesheet" type="text/css"/>
<link href="style/slider_inner.css" rel="stylesheet" type="text/css"/>
<link href="style/colorbox.css" rel="stylesheet" type="text/css"/>
<link href="style/jquery.jscrollpane.css" media="all" rel="stylesheet" type="text/css"/>
<link href="style/jquery.jscrollpane.lozenge.css" rel="stylesheet" type="text/css"/>
<link href="http://www.nativeenergy.com/assets/snippets/jot/templates/jot.css" rel="stylesheet" type="text/css"/>


[<link href="style/style.css" rel="stylesheet" type="text/css"/>,
 <link href="style/mosaic.css" rel="stylesheet" type="text/css"/>,
 <link href="style/jquery.jscrollpane.css" media="all" rel="stylesheet" type="text/css"/>,
 <link href="style/jquery.jscrollpane.lozenge.css" rel="stylesheet" type="text/css"/>,
 <link href="style/slider.css" rel="stylesheet" type="text/css"/>,
 <link href="style/colorbox.css" rel="stylesheet" type="text/css"/>,
 <link href="favicon.ico" rel="shortcut icon"/>,
 <link href="/assets/snippets/custom_gallery/logorollover/css/style.css" rel="stylesheet" type="text/css"/>]

In [108]:

def check_dp(df, output, col):
    try:
        if col in df.keys():
            dps=list(df[df.duplicated(col)].groupby(col))
            if len(dps)>0:
                for i, dp in enumerate(dps):
                    row_text, _dp = dp
                    _dp = df[df[col]==row_text]
                    row={}
                    row['TAG']=col
                    row['DUPLICATION TEXT'] = row_text
                    row['NUMBER OF DUPLICATION'] = len(_dp)
                    row['URL'] = '#\n'+'\n'.join(list(_dp['URL']))
                    output.append(row)
    except: get.pe(col)
DUPLICATIONS=[]
check_dp(DOWNLOADED, DUPLICATIONS, 'TITLE')
check_dp(DOWNLOADED, DUPLICATIONS, 'H1 1')
check_dp(DOWNLOADED, DUPLICATIONS, 'META DESCRIPTION')
check_dp(DOWNLOADED, DUPLICATIONS, 'H2 1')
get.save(get.DataFrame(DUPLICATIONS).sort_values('NUMBER OF DUPLICATION', ascending=False), 
         'DUPLICATIONS',
         columns=['TAG', 
                  'DUPLICATION TEXT',
                  'NUMBER OF DUPLICATION', 
                  'URL', ])

In [102]:
DOWNLOADED.sort_values('TITLE LENGTH', ascending=False)

7     117.0
10     46.0
1      44.0
4      44.0
3      37.0
8      37.0
2      35.0
6      35.0
5      30.0
9      29.0
11     18.0
0       NaN
Name: TITLE LENGTH, dtype: float64

In [107]:
get.save(DOWNLOADED.sort_values('TITLE LENGTH', ascending=False), 
         'DOWNLOADED',
         columns=['URL',
                    'TITLE',
                    'TITLE LENGTH',
                    'TITLE STATUS',
                    'H1-1',
                    'H1-1 LENGTH',
                    'H1-1 STATUS',
                    'FINAL URL',
                    'FINAL URL LENGTH',
                    'CANONICAL',
                    'CANONICAL LENGTH',
                    'META DESCRIPTION',
                    'META DESCRIPTION LENGTH',
                    'META DESCRIPTION STATUS',
                    'META KEYWORDS',
                    'META KEYWORDS LENGTH',
                    'H2-1',
                    'H2-1 LENGTH',
                    'H2-1 STATUS',
                    'NUMBER OF IMAGES',
                    'NUMBER OF WITH ALT-TAG',
                    'NUMBER OF WITHOUT ALT-TAG',
                    'ALT-TAG AVERAGE LENGTH',
                    'ALT-TAG MAXIMUM LENGTH',
                    'ALT-TAG MINIMUM LENGTH',
                    'NUMBER OF ALT-TAGS THAT ARE DUPLICATED',
                    'CONTENT TYPE',
                    'DATE',
                    'HEADERS',
                    'HTTP/HTTPS',
                    'REDIRECT',
                    'REDIRECT TYPE',
                    'RESPONSE TIME (MS)',
                    'STATUS',
                    'STATUS CODE'])

In [20]:
DOWNLOADED=get.DataFrame(DOWNLOADED)


In [93]:
float('1.123123', 2)

TypeError: float() takes at most 1 argument (2 given)

In [97]:
list(DOWNLOADED.keys())

['ALT-TAG AVERAGE LENGTH',
 'ALT-TAG MAXIMUM LENGTH',
 'ALT-TAG MINIMUM LENGTH',
 'CANONICAL',
 'CANONICAL LENGTH',
 'CONTENT TYPE',
 'DATE',
 'FINAL URL',
 'FINAL URL LENGTH',
 'H1-1',
 'H1-1 LENGTH',
 'H1-1 STATUS',
 'H2-1',
 'H2-1 LENGTH',
 'H2-1 STATUS',
 'H2-10',
 'H2-10 LENGTH',
 'H2-10 STATUS',
 'H2-11',
 'H2-11 LENGTH',
 'H2-11 STATUS',
 'H2-12',
 'H2-12 LENGTH',
 'H2-12 STATUS',
 'H2-13',
 'H2-13 LENGTH',
 'H2-13 STATUS',
 'H2-14',
 'H2-14 LENGTH',
 'H2-14 STATUS',
 'H2-15',
 'H2-15 LENGTH',
 'H2-15 STATUS',
 'H2-16',
 'H2-16 LENGTH',
 'H2-16 STATUS',
 'H2-17',
 'H2-17 LENGTH',
 'H2-17 STATUS',
 'H2-18',
 'H2-18 LENGTH',
 'H2-18 STATUS',
 'H2-19',
 'H2-19 LENGTH',
 'H2-19 STATUS',
 'H2-2',
 'H2-2 LENGTH',
 'H2-2 STATUS',
 'H2-3',
 'H2-3 LENGTH',
 'H2-3 STATUS',
 'H2-4',
 'H2-4 LENGTH',
 'H2-4 STATUS',
 'H2-5',
 'H2-5 LENGTH',
 'H2-5 STATUS',
 'H2-6',
 'H2-6 LENGTH',
 'H2-6 STATUS',
 'H2-7',
 'H2-7 LENGTH',
 'H2-7 STATUS',
 'H2-8',
 'H2-8 LENGTH',
 'H2-8 STATUS',
 'H2-9',
 'H2-

In [94]:
df  =get.DataFrame([{'Title':'asd', 'URL':'1'},
                    {'Title':'asd', 'URL':'2'}, 
                    {'Title':'asd', 'URL':'5'}, 
                    {'Title':'asd', 'URL':'4'} ,
                    {'Title':'asdf', 'URL':'3'} ,
                    {'Title':'asdf', 'URL':'7'},
                    {'Title':'asdf', 'URL':'9'}])

In [75]:
def get_duplications(df, col):
    dp =  df[df.duplicated(col)]
    if len(dp) >0:
        return len(dp), '\n'.join(list(dp['URL'])), dp.iloc[0][col]
    return 0, None, None

In [114]:
title, col=list(df[df.duplicated('Title')].groupby('Title'))[0]

In [139]:
def check_dp(df, col):
    dps=list(df[df.duplicated(col)].groupby(col))
    row, dp = dps[0]
    dp = df[df[col]==row]
    return len(dp), '\n'.join(list(dp['URL'])), row

In [140]:
check_dp(get.DataFrame(DOWNLOADED), 'H1 0')

(3,
 'https://nativeenergy.com/\nhttps://nativeenergy.com/customer-service1.html\nhttps://nativeenergy.com/site-map.html',
 'Your Climate Solutions Expert: carbon offsets, renewable energy credits, and carbon tracking services')

In [118]:
df[df['Title']==title]

,Title,URL
0,asd,1
1,asd,2
2,asd,5
3,asd,4


In [99]:
df[df.duplicated('Title')].groupby('Title').count()

,URL
Title,
asd,3
asdf,2


In [165]:
tags=imgs
row['Number of images'.upper()] = len(tags)
imgs_with_tag=[img for img in tags if img.has_attr('alt')]

row['Number of with alt-tag'.upper()] = len(imgs_with_tag)
row['Number of without alt-tag'.upper()] = len(tags) - len(imgs_with_tag)
alt_tags=numpy.array([len(img['alt']) for img in imgs_with_tag])
if len(alt_tags)>0:
    row['alt-tag Minimum length'.upper()] = alt_tags.min()
    row['alt-tag Average length'.upper()] = alt_tags.mean()
    row['alt-tag Maximum length'.upper()] = alt_tags.max()
df = get.DataFrame([img['alt'] for img in imgs_with_tag])
row['Number of Alt-Tags that are duplicated'.upper()] = len(df[df.duplicated])

In [167]:
ROW_ERROR

{'CONTENT TYPE': 'image/jpeg',
 'DATE': 'Thu, 09 Nov 2017 07:00:49 GMT',
 'DEPTH': 5,
 'FINAL URL': 'https://www.mobiledetailingpros.com/wp-content/uploads/2017/04/SUV-product.jpg',
 'FINAL URL LENGTH': 78,
 'FOLDER LEVEL 1': 'wp-content',
 'FOLDER LEVEL 2': 'uploads',
 'FOLDER LEVEL 3': '2017',
 'FOLDER LEVEL 4': '04',
 'HEADERS': {'Date': 'Thu, 09 Nov 2017 07:00:49 GMT', 'Server': 'Apache/2.4.25 (Debian)', 'Last-Modified': 'Thu, 28 Sep 2017 05:15:56 GMT', 'ETag': '"210fa-55a3900c37b00"', 'Accept-Ranges': 'bytes', 'Content-Length': '135418', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'image/jpeg'},
 'HTTP/HTTPS': 'HTTPS',
 'NUMBER OF IMAGES': 0,
 'NUMBER OF WITH ALT-TAG': 0,
 'NUMBER OF WITHOUT ALT-TAG': 0,
 'REDIRECT': False,
 'REDIRECT TYPE': 'NONE',
 'RESPONSE TIME (MS)': 376.627,
 'SOUP': ����#�Exif  MM *    
      �      8         �                            �       �(       1    $   �2       ؇i 

In [76]:
get_duplications(df, 'Title')

(1, '2', 'asd')

In [68]:
get_duplications(get.DataFrame(DOWNLOADED), 'META DESCRIPTION')

(1, 'https://nativeenergy.com/site-map.html', '[Missing meta description]')

In [69]:
get_duplications(get.DataFrame(DOWNLOADED), 'TITLE')

(0, None, None)

In [65]:
get_duplications(get.DataFrame(DOWNLOADED), 'H1 0')

(2,
 ['https://nativeenergy.com/customer-service1.html',
  'https://nativeenergy.com/site-map.html'],
 'H1 0',
 'Your Climate Solutions Expert: carbon offsets, renewable energy credits, and carbon tracking services')

In [62]:
get_duplications(get.DataFrame(DOWNLOADED), 'URL')

(0, None, None)

In [63]:
get_duplications(get.DataFrame(DOWNLOADED), 'H2 0')

KeyError: 'H2 0'